In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/amazon_ml/

/content/drive/MyDrive/amazon_ml


In [ ]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.merge import concatenate
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import scipy
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from random import shuffle
import gc

import os
import codecs

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from string import punctuation

from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

import sys
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
BASE_DIR = ''
EMBEDDING_FILE = BASE_DIR + 'GoogleNews-vectors-negative300.bin'
TRAIN_DATA_FILE = BASE_DIR + 'dataset/train.csv'
TEST_DATA_FILE = BASE_DIR + 'dataset/test.csv'
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2
TEMP_PATH = 'prepro_data2/'

# PREPROCESSING

In [ ]:
# not needed
print('Indexing word vectors')
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

print('Processing text dataset')
stops = set(stopwords.words("english"))
w2v_set = set(word2vec.vocab)

Indexing word vectors
Found 3000000 word vectors of word2vec
Processing text dataset


In [ ]:
# not needed
train_df = pd.read_csv(TRAIN_DATA_FILE,escapechar='\\',quoting=csv.QUOTE_NONE)
print(train_df.shape)

test_df = pd.read_csv(TEST_DATA_FILE,escapechar='\\',quoting=csv.QUOTE_NONE)
print(test_df.shape)

(2903024, 5)
(110775, 5)


In [ ]:
# checkpointed for 3 columns - not needed
def text_to_wordlist(text, remove_stopwords=False, stem_words=False):
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)

    text = re.sub(r'[^\w\s]',' ',text.lower().replace('_',' '))
    text = re.sub(r'(\b(\w*)_(\w*)\b)|(\b(\w*)(\d+)(\w*)\b)|\b(\w*)[^\w\s](\w*)\b','',text)

    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    text = text.split()
    if remove_stopwords:
        text = [w for w in text if w not in stops]
    text = [w for w in text if w in w2v_set and len(w)>2]
    text = " ".join(text)
    text = text.split()
    text = ' '.join(text)
    return(text)

for col in ['TITLE','DESCRIPTION','BULLET_POINTS']:
  train_df[col].fillna('',inplace=True)
  train_df[col] = train_df[col].apply(lambda x:text_to_wordlist(x,remove_stopwords=True, stem_words=False))
  test_df[col].fillna('',inplace=True)
  test_df[col] = test_df[col].apply(lambda x:text_to_wordlist(x,remove_stopwords=True, stem_words=False))

train_df.to_csv(TEMP_PATH+'train_data_cleaned.csv',index=False)
test_df.to_csv(TEMP_PATH+'test_data_cleaned.csv',index=False)

# CLEANED DATA

In [ ]:
train_df = pd.read_csv(TEMP_PATH+'train_data_cleaned.csv')
test_df = pd.read_csv(TEMP_PATH+'test_data_cleaned.csv')

In [ ]:
train_df.fillna('',inplace=True)
test_df.fillna('',inplace=True)

In [ ]:
for col in ['TITLE','DESCRIPTION','BULLET_POINTS']:
  train_df['LEN_'+col] = train_df[col].apply(lambda x:len(x.split()))
  test_df['LEN_'+col] = test_df[col].apply(lambda x:len(x.split()))
  print(col,' done')

TITLE  done
DESCRIPTION  done
BULLET_POINTS  done


In [ ]:
print(np.percentile(train_df.loc[train_df['LEN_TITLE']>0,'LEN_TITLE'].values,99))
print(np.percentile(train_df.loc[train_df['LEN_DESCRIPTION']>0,'LEN_DESCRIPTION'].values,90))
print(np.percentile(train_df.loc[train_df['LEN_BULLET_POINTS']>0,'LEN_BULLET_POINTS'].values,90))
print()
print(np.percentile(test_df.loc[test_df['LEN_TITLE']>0,'LEN_TITLE'].values,99))
print(np.percentile(test_df.loc[test_df['LEN_DESCRIPTION']>0,'LEN_DESCRIPTION'].values,90))
print(np.percentile(test_df.loc[test_df['LEN_BULLET_POINTS']>0,'LEN_BULLET_POINTS'].values,90))

25.0
125.0
88.0

25.0
132.0
98.0


In [ ]:
BRACKETS = [25,125,90]

def final_cleaner(x,brack):
  x = x.split()
  if len(x)>brack:
    res = []
    [res.append(i) for i in x if i not in res]
    res = res[:brack]
    return ' '.join(res)
  else:
    return ' '.join(x)

for i, col in enumerate(['TITLE','DESCRIPTION','BULLET_POINTS']):
  train_df[col] = train_df[col].apply(lambda x:final_cleaner(x,BRACKETS[i]))
  test_df[col] = test_df[col].apply(lambda x:final_cleaner(x,BRACKETS[i]))
  print(col,' done')

TITLE  done
DESCRIPTION  done
BULLET_POINTS  done


In [ ]:
for col in ['TITLE','DESCRIPTION','BULLET_POINTS']:
  train_df['LEN_'+col] = train_df[col].apply(lambda x:len(x.split()))
  test_df['LEN_'+col] = test_df[col].apply(lambda x:len(x.split()))
  print(col,' done')

TITLE  done
DESCRIPTION  done
BULLET_POINTS  done


In [ ]:
print(np.percentile(train_df.loc[train_df['LEN_TITLE']>0,'LEN_TITLE'].values,100))
print(np.percentile(train_df.loc[train_df['LEN_DESCRIPTION']>0,'LEN_DESCRIPTION'].values,100))
print(np.percentile(train_df.loc[train_df['LEN_BULLET_POINTS']>0,'LEN_BULLET_POINTS'].values,100))
print()
print(np.percentile(test_df.loc[test_df['LEN_TITLE']>0,'LEN_TITLE'].values,100))
print(np.percentile(test_df.loc[test_df['LEN_DESCRIPTION']>0,'LEN_DESCRIPTION'].values,100))
print(np.percentile(test_df.loc[test_df['LEN_BULLET_POINTS']>0,'LEN_BULLET_POINTS'].values,100))

25.0
125.0
90.0

25.0
125.0
90.0


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(np.squeeze(np.vstack([train_df['TITLE'].values[:,np.newaxis],test_df['TITLE'].values[:,np.newaxis],
                                             train_df['DESCRIPTION'].values[:,np.newaxis],test_df['DESCRIPTION'].values[:,np.newaxis],
                                             train_df['BULLET_POINTS'].values[:,np.newaxis],test_df['BULLET_POINTS'].values[:,np.newaxis]])))
print(len(tokenizer.word_index))

In [ ]:
if os.path.exists(TEMP_PATH):
  pass
else:
  os.makedirs(TEMP_PATH,exist_ok=True)

In [ ]:
for i,col in enumerate(['TITLE','DESCRIPTION','BULLET_POINTS']):
  data = tokenizer.texts_to_sequences(train_df[col].values)
  data_test = tokenizer.texts_to_sequences(test_df[col].values)
  data = pad_sequences(data, maxlen=BRACKETS[i])
  data_test = pad_sequences(data_test, maxlen=BRACKETS[i])

  print('Shape of data:', data.shape)
  print('Shape of data_test:', data_test.shape)

  with open(TEMP_PATH+'data_{}.npy'.format(i), 'wb') as f:
      np.save(f, data)
  with open(TEMP_PATH+'data_test_{}.npy'.format(i), 'wb') as f:
      np.save(f, data_test)

enc = LabelEncoder()
Y = train_df['BROWSE_NODE_ID'].values
Y = enc.fit_transform(Y)
print(Y.shape)
print(enc.classes_.shape)

with open(TEMP_PATH+'labels.npy', 'wb') as f:
    np.save(f, Y)
with open(TEMP_PATH+'labels_mapping.npy', 'wb') as f:
    np.save(f, enc.classes_)

Shape of data: (2903024, 25)
Shape of data_test: (110775, 25)
Shape of data: (2903024, 125)
Shape of data_test: (110775, 125)
Shape of data: (2903024, 90)
Shape of data_test: (110775, 90)
(2903024,)
(9919,)


In [ ]:
del data
del data_test
del Y
del enc
gc.collect()

171

In [ ]:
del train_df
del test_df
del train_df_col
del test_df_col
gc.collect()

87

In [ ]:
print('Indexing word vectors')
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
print('Found %s word vectors of word2vec' % len(word2vec.vocab))

Indexing word vectors
Found 3000000 word vectors of word2vec


In [ ]:
nb_words = len(tokenizer.word_index)+1
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
      embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
print(embedding_matrix.shape)

Null word embeddings: 1
(90766, 300)


In [ ]:
with open(TEMP_PATH+'embeddings.npy', 'wb') as f:
    np.save(f, embedding_matrix)

In [ ]:
with open(TEMP_PATH+'data_0.npy', 'rb') as f:
    data_0 = np.load(f)

with open(TEMP_PATH+'data_test_0.npy', 'rb') as f:
    data_test_0 = np.load(f)

with open(TEMP_PATH+'data_1.npy', 'rb') as f:
    data_1 = np.load(f)

with open(TEMP_PATH+'data_test_1.npy', 'rb') as f:
    data_test_1 = np.load(f)

with open(TEMP_PATH+'data_2.npy', 'rb') as f:
    data_2 = np.load(f)

with open(TEMP_PATH+'data_test_2.npy', 'rb') as f:
    data_test_2 = np.load(f)

data = np.concatenate([data_0, data_1, data_2], axis=1)
print(data.shape)
data_test = np.concatenate([data_test_0, data_test_1, data_test_2], axis=1)
print(data_test.shape)

with open(TEMP_PATH+'data.npy', 'wb') as f:
    np.save(f, data)
with open(TEMP_PATH+'data_test.npy', 'wb') as f:
    np.save(f, data_test)

(2903024, 240)
(110775, 240)


# LOADING PREPROCESSED
* Start from here
* 58005, 2903024, 110775, 300, 82, 9919

In [ ]:
# loading preproceesed

with open(TEMP_PATH+'embeddings.npy', 'rb') as f:
    embedding_matrix = np.load(f)

with open(TEMP_PATH+'labels.npy', 'rb') as f:
    Y = np.load(f)

with open(TEMP_PATH+'labels_mapping.npy', 'rb') as f:
    enc_classes = np.load(f)

with open(TEMP_PATH+'data.npy', 'rb') as f:
    data = np.load(f)

with open(TEMP_PATH+'data_test.npy', 'rb') as f:
    data_test = np.load(f)

print(embedding_matrix.shape)
print(Y.shape)
print(enc_classes.shape)
print(data.shape)
print(data_test.shape)

(90766, 300)
(2903024,)
(9919,)
(2903024, 240)
(110775, 240)


In [ ]:
MODEL_NUM = 6
nb_words = embedding_matrix.shape[0]
NUM_LABELS = len(enc_classes)
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM = embedding_matrix.shape[1]
MAX_SEQUENCE_LENGTH = 240
NUM_EPOCHS = 20
BATCH_SIZE = 1024

os.makedirs(f'model-{MODEL_NUM}',exist_ok=True)
MODEL_PATH = f'model-{MODEL_NUM}/'

In [ ]:
from keras.layers import Layer
import keras.backend as K

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(101)
perm = np.random.permutation(len(data))
idx_train = perm[:int(len(data)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data)*(1-VALIDATION_SPLIT)):]
print(len(idx_train), len(idx_val))

data_train = data[idx_train]
data_val = data[idx_val]
Y_train = Y[idx_train]
Y_val = Y[idx_val]
del data
del Y
gc.collect()

# model
embedding_layer = Embedding(nb_words,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False)

lstm_layer_1 = LSTM(250, return_sequences=True, dropout=0.15, recurrent_dropout=0.15)
#lstm_layer_2 = LSTM(200, return_sequences=False, dropout=0.15, recurrent_dropout=0.15)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
x1 = embedding_layer(sequence_input)
x1 = lstm_layer_1(x1)
x1 = attention()(x1)
x1 = Dropout(0.10)(x1)
x1 = BatchNormalization()(x1)
x1 = Dense(10000, activation='relu')(x1)
x1 = Dropout(0.10)(x1)
x1 = BatchNormalization()(x1)
preds = Dense(NUM_LABELS, activation='softmax')(x1)

model = Model(inputs=sequence_input, outputs=preds)
print(model.summary())

2322419 580605
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 240)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 240, 300)          27229800  
_________________________________________________________________
lstm (LSTM)                  (None, 240, 250)          551000    
_________________________________________________________________
attention (attention)        (None, 250)               490       
_________________________________________________________________
dropout (Dropout)            (None, 250)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 250)               1000      
_________________________________________________________________
dense (Dense)                (None, 10000)    

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((data_train, Y_train))
train_dataset = train_dataset.shuffle(buffer_size=data_train.shape[0]).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((data_val, Y_val))
val_dataset = val_dataset.batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices(data_test)
test_dataset = test_dataset.batch(BATCH_SIZE)

loss_fn = keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
model.load_weights(MODEL_PATH+'epoch:01-loss:1.046-val_acc:0.737.h5')

In [ ]:
for epoch in range(NUM_EPOCHS):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            pred = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, pred)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        train_acc_metric.update_state(y_batch_train, pred)
        if step % 20 == 0:
            print("Step-->{:02d}, Loss-->{:.3f}, Accuracy-->{:.3f}".format(step+1, loss_value, train_acc_metric.result().numpy()))

    train_acc = train_acc_metric.result()
    print("Training acc: %.4f" % (float(train_acc)))
    train_acc_metric.reset_states()

    for x_batch_val, y_batch_val in val_dataset:
        val_pred = model(x_batch_val, training=False)
        val_acc_metric.update_state(y_batch_val, val_pred)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))

    ans = []
    for x_batch_test in test_dataset:
        test_pred = model(x_batch_test, training=False).numpy().argmax(axis=1)
        ans.append(test_pred)
    ans = np.concatenate(ans)
    ans = enc_classes[ans]
    sub_df = pd.DataFrame(ans, columns=['BROWSE_NODE_ID'])
    sub_df['PRODUCT_ID'] = sub_df.index + 1
    sub_df = sub_df[['PRODUCT_ID','BROWSE_NODE_ID']]
    sub_df.to_csv(MODEL_PATH+'epoch:{:02d}-val_acc:{:.3f}-sub.csv'.format(epoch,val_acc), index=False)
    print("Output saved for test set.")
    model.save_weights(MODEL_PATH+"epoch:{:02d}-loss:{:.3f}-val_acc:{:.3f}.h5".format(epoch, loss_value.numpy(), float(val_acc)))